📋 Cell 1: Setup & Installation

# 🥕 Carrot Price Prediction AI Agent (ENHANCED VERSION)

## 🆕 What's New in This Version:

### ✅ **General Knowledge Base Added**
The agent now includes comprehensive agricultural economics knowledge:
- **Weather-Price Relationships:** How rainfall affects prices (7-14 day lags, threshold effects)
- **Fuel Price Impacts:** Transportation cost correlations with market prices
- **Seasonal Patterns:** Peak/low production periods, volatility windows
- **Supply Dynamics:** Regional production, harvest cycles, supply disruptions
- **Demand Patterns:** Festival effects, weekend demand, market closure impacts
- **Price Triggers:** Specific factors causing increases/decreases

### ✅ **Original Dataset Integration**
Can now analyze your full historical dataset:
- Weather data from 11 meteorological stations
- Fuel prices (Diesel LAD/LSD, Petrol LP95/LP92)
- Supply data from multiple growing regions
- Market demand indicators
- All 163+ engineered features

### ✅ **Smarter Context Building**
- Automatically detects question type (why/what/how/compare)
- Adds relevant knowledge based on query intent
- Combines specific data with general market understanding
- Provides educated explanations even without exact date data

### 🎯 **Problem Solved:**
**Before:** "I don't have information for April 2-8, 2024" ❌  
**Now:** "Based on typical patterns and available data, prices likely increased due to..." ✅

### 💡 **Use Cases:**
1. **With Predictions Only:** Agent uses general knowledge to explain trends
2. **With Original Dataset:** Agent provides specific data-driven explanations
3. **Historical Analysis:** "Why did X happen?" gets detailed weather/fuel/supply context
4. **Research Questions:** Methodology, feature engineering, model comparisons
5. **Market Education:** General agricultural economics questions

---

In [40]:
# Install required packages
!pip install -q groq gradio pandas numpy scikit-learn

print("✅ Packages installed!")
print("Using Groq API (FREE) with Llama 3.1 70B model")

✅ Packages installed!
Using Groq API (FREE) with Llama 3.1 70B model


📋 Cell 2: Configuration

In [ ]:
from groq import Groq
import gradio as gr
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re


# Initialize Groq client
groq_client = Groq(api_key=GROQ_API_KEY)

print("="*60)
print("✅ Groq API Client Initialized!")
print("Model: Llama 3.1 70B (FREE)")
print("="*60)

✅ Groq API Client Initialized!
Model: Llama 3.1 70B (FREE)


📋 Cell 3: Load Your LSTM Predictions & Original Dataset

In [ ]:
# Load your LSTM predictions AND original dataset
print("="*60)
print("📊 LOADING PREDICTION DATA & ORIGINAL DATASET")
print("="*60)

# Load LSTM predictions
try:
    predictions_df = pd.read_csv('lstm_predictions.csv')
    predictions_df['date'] = pd.to_datetime(predictions_df['date'])
    print(f"✅ Loaded {len(predictions_df)} predictions from CSV")
    print(f"Date range: {predictions_df['date'].min()} to {predictions_df['date'].max()}")
    print("\nFirst few rows:")
    print(predictions_df.head())

except FileNotFoundError:
    print("⚠️ Predictions CSV not found. Creating sample data for testing...")
    dates = pd.date_range('2024-01-01', periods=180, freq='D')
    np.random.seed(42)
    predictions_df = pd.DataFrame({
        'date': dates,
        'actual_price': np.random.randint(120, 350, 180),
        'predicted_price': np.random.randint(110, 360, 180),
    })
    predictions_df['error'] = predictions_df['predicted_price'] - predictions_df['actual_price']
    predictions_df['mape'] = np.abs(predictions_df['error'] / predictions_df['actual_price']) * 100
    print(f"✅ Created {len(predictions_df)} sample predictions")

# Load ORIGINAL DATASET with all features
original_df = None
try:
    # Try to load your original dataset with weather, fuel, supply, demand data
    original_df = pd.read_csv('carrot_price_dataset.csv')  # or your actual filename
    original_df['date'] = pd.to_datetime(original_df['date'])
    print(f"\n✅ Loaded ORIGINAL DATASET: {len(original_df)} records")
    print(f"Date range: {original_df['date'].min()} to {original_df['date'].max()}")
    print(f"Columns: {len(original_df.columns)} features")
    print(f"Features: {', '.join(original_df.columns[:10])}...")  # Show first 10 columns
    
except FileNotFoundError:
    print("\n⚠️ Original dataset not found. Please upload your full dataset CSV.")
    print("📌 Upload file with name: 'carrot_price_dataset.csv'")
    print("   This should include: prices, weather, fuel, supply, demand data")

print("\n" + "="*60)

📊 LOADING PREDICTION DATA
⚠️ CSV file not found. Creating sample data for testing...
✅ Created 180 sample predictions
📌 Remember to upload your actual LSTM predictions CSV!

Sample data preview:
        date  actual_price  predicted_price  error        mape
0 2024-01-01           222              241     19    8.558559
1 2024-01-02           299              331     32   10.702341
2 2024-01-03           212              338    126   59.433962
3 2024-01-04           134              260    126   94.029851
4 2024-01-05           226              340    114   50.442478
5 2024-01-06           191              346    155   81.151832
6 2024-01-07           308              252    -56   18.181818
7 2024-01-08           140              280    140  100.000000
8 2024-01-09           222              138    -84   37.837838
9 2024-01-10           241              145    -96   39.834025



📋 Cell 4: Agent Core Logic

📋 Cell 3.5: Upload Original Dataset (IMPORTANT!)

**To get the BEST performance, upload your original dataset:**

1. Your dataset should include:
   - Date column
   - Carrot price data (actual historical prices)
   - Weather data (precipitation from 11 stations)
   - Fuel prices (diesel LAD/LSD, petrol LP95/LP92)
   - Supply data (from growing regions)
   - Demand indicators (market status, trading activity)

2. Save the file as: `carrot_price_dataset.csv`

3. Upload it to this Colab notebook

**Why upload the original dataset?**
- Agent can analyze ACTUAL weather, fuel, supply data for any date range
- Provides context for explaining WHY prices changed
- Enables deeper insights: "On April 5, heavy rainfall (145mm) in Nuwara Eliya caused supply disruption"
- Much better than just having predictions alone!

**Note:** Even without the original dataset, the agent now has general agricultural knowledge to explain price movements!

In [ ]:
class CarrotPriceAgent:
    """AI Agent for Carrot Price Predictions using Groq API with General Knowledge"""

    def __init__(self, groq_client, predictions_df, original_df=None):
        self.groq = groq_client
        self.predictions = predictions_df
        self.original_data = original_df  # Full dataset with all features

        # Model comparison results - UPDATED WITH ACTUAL RESULTS
        self.model_results = {
            'Simple LSTM (Best)': {
                'MAPE': 19.93,
                'MAE': 58.87,
                'RMSE': 84.05,
                'R2': 0.8651
            },
            'Bidirectional LSTM': {
                'MAPE': 21.46,
                'MAE': 69.89,
                'RMSE': 102.04,
                'R2': 0.8011
            },
            'Univariate LSTM': {
                'MAPE': 21.90,
                'MAE': 66.01,
                'RMSE': 136.82,
                'R2': 0.6428
            },
            'Random Forest Tuned': {
                'MAPE': 34.10,
                'MAE': 123.43,
                'RMSE': 178.08,
                'R2': 0.3931
            },
            'ARIMAX': {
                'MAPE': 88.80,
                'MAE': 293.54,
                'RMSE': 363.46,
                'R2': -0.15
            }
        }

        # GENERAL KNOWLEDGE BASE - Agricultural Economics & Market Dynamics
        self.general_knowledge = """
=== CARROT PRICE DYNAMICS IN SRI LANKA ===

**TYPICAL PRICE RANGES (Dambulla Market):**
- Normal range: Rs. 120 - 250 per kg
- High price events: Rs. 300 - 450 per kg
- Low price events: Rs. 50 - 100 per kg

**MAJOR GROWING REGIONS:**
1. Central Highlands: Nuwara Eliya, Kandapola, Ragala, Thalawakale, Pussellawa, Hanguranketha
2. Uva Province: Bandarawela, Walimada
3. Northern Region: Jaffna

**SEASONAL PATTERNS:**
- Peak production: December - February (cooler weather)
- Low production: June - August (monsoon season)
- Price volatility highest during: March-May, September-November

**WEATHER IMPACTS (Research Findings):**
1. **Rainfall Effects:**
   - Moderate rainfall (50-100mm): Positive for crop growth → Lower prices
   - Heavy rainfall (>150mm): Crop damage, transportation issues → Higher prices
   - Drought conditions: Reduced yields → Higher prices
   - Weather lag: 7-14 days from rainfall to market impact

2. **Central Highland Precipitation:**
   - Explains 12% of price variance
   - 1% precipitation increase → ~2.3% price decrease (under normal conditions)
   - Critical growing regions most sensitive to weather changes

**FUEL PRICE IMPACTS:**
- Transportation costs: 15-20% of final market price
- Diesel price correlation: Strong positive (r=0.65)
- Fuel price spikes → 3-5 day lag → Market price increase
- 2022 fuel crisis: Prices surged 45% due to transportation cost shock

**SUPPLY FACTORS:**
- Market supply from 11 major growing regions
- Supply disruptions → 30-50% price spikes within 2-3 days
- Oversupply from multiple regions → 20-30% price drops
- Harvest cycles: 90-120 days from planting to market

**DEMAND PATTERNS:**
- Weekend demand: 15-20% higher than weekdays
- Festival seasons (Sinhala/Tamil New Year, Vesak): 25-35% demand spike
- Market closed days: Next-day price volatility increases
- Urban consumption peaks: Wednesday-Saturday

**PRICE INCREASE TRIGGERS:**
1. Heavy rainfall in Nuwara Eliya/Bandarawela (7-14 day lag)
2. Fuel price increases (3-5 day lag)
3. Supply disruptions from major growing regions
4. Festival season demand surge
5. Market closure (accumulation effect)
6. Transportation strikes/disruptions
7. Adverse weather during harvest period

**PRICE DECREASE TRIGGERS:**
1. Good weather → Abundant harvest
2. Multiple regions harvesting simultaneously (oversupply)
3. Fuel price stabilization/decrease
4. Low demand periods (post-festival)
5. Improved transportation infrastructure
6. Market intervention/buffer stock release

**VOLATILITY PATTERNS:**
- Normal daily volatility: ±5-8%
- High volatility events: ±15-25%
- Crisis periods (2022): ±30-45%
- Volatility highest: March-May, September-November

**RESEARCH INSIGHTS - Feature Importance:**
1. Price Features (48.7% importance): Historical lags, rolling averages
2. Weather Features (19.2% importance): Central Highland precipitation most critical
3. Market Demand (14.5% importance): Trading activity, market status
4. Supply Factors (8.9% importance): Regional production levels
5. Fuel Prices (6.1% importance): Diesel LAD, Petrol LP95
6. Temporal Features (2.6% importance): Day of week, month, seasonality

**MODEL ARCHITECTURE (Simple LSTM - Best Performer):**
- Features: 9 carefully selected from 163 engineered features
- Architecture: Single LSTM layer (50 units) + Dense(25) + Dense(1)
- Regularization: Dropout(0.2), L2(0.001)
- Training: Early stopping at epoch 52/67
- Generalization gap: Only 5.78% (train 14.15% → test 19.93%)

**ABLATION STUDY FINDINGS:**
- Removing price features: +8.3% MAPE increase (most critical)
- Removing weather features: +3.1% MAPE increase
- Removing demand features: +2.4% MAPE increase
- All feature categories contribute meaningfully

**DATA SOURCES:**
1. Price data: Dambulla Economic Center (daily records 2020-2025)
2. Weather data: Department of Meteorology (11 stations)
3. Fuel prices: Ceylon Petroleum Corporation
4. Supply data: Department of Agriculture regional offices
5. Market status: Dambulla market operational records
"""

        # Data sources description
        self.data_sources = """
DATA COLLECTION METHODOLOGY:
- Time period: January 2020 - July 2025 (2,017 daily observations)
- Primary market: Dambulla wholesale market (largest vegetable market in Sri Lanka)
- Initial features: 289 engineered features across 6 categories
- LSTM features: 163 engineered features after domain-specific engineering
- Final features: 9 features after 4-stage selection pipeline
- Data quality: Cleaned, imputed missing values, outlier detection applied
"""

    def extract_dates_from_query(self, question):
        """Extract dates from natural language question"""
        # Pattern 1: YYYY-MM-DD format
        dates = re.findall(r'\d{4}-\d{2}-\d{2}', question)
        if dates:
            return dates

        # Pattern 2: Month names with dates
        month_patterns = re.findall(r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2})(?:-(\d{1,2}))?(?:,?\s+(\d{4}))?', question, re.IGNORECASE)
        if month_patterns:
            return month_patterns

        return []

    def get_price_for_date(self, date_str):
        """Get prediction for specific date"""
        try:
            target_date = pd.to_datetime(date_str)
            row = self.predictions[self.predictions['date'] == target_date]

            if len(row) == 0:
                return None

            return {
                'date': date_str,
                'actual': float(row['actual_price'].iloc[0]),
                'predicted': float(row['predicted_price'].iloc[0]),
                'error': float(row['error'].iloc[0]),
                'mape': float(row.get('mape', [0]).iloc[0]) if 'mape' in row.columns else None
            }
        except Exception as e:
            print(f"Error getting price for {date_str}: {e}")
            return None

    def get_original_data_for_date_range(self, start_date, end_date):
        """Get original dataset features for date range (weather, fuel, supply, demand)"""
        if self.original_data is None:
            return None
        
        try:
            start = pd.to_datetime(start_date)
            end = pd.to_datetime(end_date)
            
            mask = (self.original_data['date'] >= start) & (self.original_data['date'] <= end)
            filtered = self.original_data[mask]
            
            if len(filtered) == 0:
                return None
            
            # Extract key insights from the period
            insights = {
                'date_range': f"{start_date} to {end_date}",
                'days': len(filtered),
                'price_start': filtered.iloc[0]['price'] if 'price' in filtered.columns else None,
                'price_end': filtered.iloc[-1]['price'] if 'price' in filtered.columns else None,
                'price_change': filtered.iloc[-1]['price'] - filtered.iloc[0]['price'] if 'price' in filtered.columns else None,
                'avg_price': filtered['price'].mean() if 'price' in filtered.columns else None,
                'price_volatility': filtered['price'].std() if 'price' in filtered.columns else None,
            }
            
            # Add weather insights if available
            weather_cols = [col for col in filtered.columns if 'precipitation' in col.lower() or 'rainfall' in col.lower()]
            if weather_cols:
                insights['avg_rainfall'] = filtered[weather_cols].mean().mean()
                insights['heavy_rain_days'] = (filtered[weather_cols].mean(axis=1) > 100).sum()
            
            # Add fuel price insights if available
            fuel_cols = [col for col in filtered.columns if 'diesel' in col.lower() or 'petrol' in col.lower()]
            if fuel_cols:
                insights['avg_fuel_price'] = filtered[fuel_cols].mean().mean()
                insights['fuel_price_change'] = filtered[fuel_cols].iloc[-1].mean() - filtered[fuel_cols].iloc[0].mean()
            
            # Add supply insights if available
            supply_cols = [col for col in filtered.columns if 'supply' in col.lower() or 'quantity' in col.lower()]
            if supply_cols:
                insights['avg_supply'] = filtered[supply_cols].mean().mean()
            
            return insights
            
        except Exception as e:
            print(f"Error getting original data: {e}")
            return None

    def get_date_range_data(self, start_date, end_date):
        """Get predictions for date range"""
        try:
            start = pd.to_datetime(start_date)
            end = pd.to_datetime(end_date)

            mask = (self.predictions['date'] >= start) & (self.predictions['date'] <= end)
            filtered = self.predictions[mask]

            if len(filtered) == 0:
                return None

            return {
                'count': len(filtered),
                'avg_actual': filtered['actual_price'].mean(),
                'avg_predicted': filtered['predicted_price'].mean(),
                'avg_error': filtered['error'].mean(),
                'price_change': filtered['actual_price'].iloc[-1] - filtered['actual_price'].iloc[0],
                'price_change_pct': ((filtered['actual_price'].iloc[-1] - filtered['actual_price'].iloc[0]) / filtered['actual_price'].iloc[0]) * 100,
                'volatility': filtered['actual_price'].std(),
                'max_price': filtered['actual_price'].max(),
                'min_price': filtered['actual_price'].min(),
            }
        except Exception as e:
            print(f"Error getting range data: {e}")
            return None

    def build_context(self, question):
        """Build relevant context for the LLM with general knowledge"""
        context = "You are an AI assistant for a carrot price prediction research project.\n\n"

        question_lower = question.lower()

        # ALWAYS ADD GENERAL KNOWLEDGE for "why" questions
        if any(word in question_lower for word in ['why', 'reason', 'cause', 'explain', 'increase', 'decrease', 'spike', 'drop', 'change']):
            context += self.general_knowledge + "\n\n"

        # Add data sources for research questions
        if any(word in question_lower for word in ['data', 'source', 'where', 'research', 'collect', 'methodology', 'how']):
            context += self.data_sources + "\n\n"

        # Add model comparison for model questions
        if any(word in question_lower for word in ['model', 'arima', 'lstm', 'random forest', 'compare', 'better', 'best', 'performance', 'accuracy']):
            context += "MODEL PERFORMANCE COMPARISON:\n\n"
            for model, metrics in sorted(self.model_results.items(), key=lambda x: x[1]['MAPE']):
                context += f"{model}:\n"
                context += f"  - Test MAPE: {metrics['MAPE']:.2f}%\n"
                context += f"  - Test MAE: Rs. {metrics['MAE']:.2f}\n"
                context += f"  - Test RMSE: Rs. {metrics['RMSE']:.2f}\n"
                context += f"  - R² Score: {metrics['R2']:.4f}\n\n"

            best_model = min(self.model_results.items(), key=lambda x: x[1]['MAPE'])
            context += f"Best Performing Model: {best_model[0]} (MAPE: {best_model[1]['MAPE']:.2f}%)\n\n"

        # Add price data for prediction questions
        if any(word in question_lower for word in ['price', 'predict', 'forecast', 'cost', 'value', '2024', '2025']):
            dates = self.extract_dates_from_query(question)

            if dates:
                # Try to get original dataset information for the period
                if len(dates) >= 2:
                    original_insights = self.get_original_data_for_date_range(dates[0], dates[1])
                    if original_insights:
                        context += f"ACTUAL DATA FOR PERIOD {original_insights['date_range']}:\n"
                        if original_insights['price_start']:
                            context += f"  - Starting Price: Rs. {original_insights['price_start']:.2f}\n"
                        if original_insights['price_end']:
                            context += f"  - Ending Price: Rs. {original_insights['price_end']:.2f}\n"
                        if original_insights['price_change']:
                            change_pct = (original_insights['price_change'] / original_insights['price_start']) * 100
                            context += f"  - Price Change: Rs. {original_insights['price_change']:.2f} ({change_pct:+.1f}%)\n"
                        if original_insights.get('avg_rainfall'):
                            context += f"  - Avg Rainfall: {original_insights['avg_rainfall']:.1f}mm\n"
                        if original_insights.get('heavy_rain_days'):
                            context += f"  - Heavy Rain Days: {original_insights['heavy_rain_days']}\n"
                        if original_insights.get('fuel_price_change'):
                            context += f"  - Fuel Price Change: Rs. {original_insights['fuel_price_change']:+.2f}\n"
                        context += "\n"
                
                # Get specific date predictions
                for date in dates[:3]:  # Max 3 dates
                    if isinstance(date, str):
                        price_data = self.get_price_for_date(date)
                        if price_data:
                            context += f"PRICE DATA FOR {date}:\n"
                            context += f"  - Actual Price: Rs. {price_data['actual']:.2f}\n"
                            context += f"  - LSTM Predicted: Rs. {price_data['predicted']:.2f}\n"
                            context += f"  - Prediction Error: Rs. {price_data['error']:.2f}\n"
                            if price_data['mape']:
                                context += f"  - Prediction Accuracy: {100 - price_data['mape']:.2f}%\n"
                            context += "\n"
            else:
                # No specific date, show recent trends
                recent = self.predictions.tail(7)
                context += "RECENT PRICE TRENDS (Last 7 days):\n"
                for _, row in recent.iterrows():
                    context += f"  {row['date'].strftime('%Y-%m-%d')}: Actual=Rs.{row['actual_price']:.0f}, Predicted=Rs.{row['predicted_price']:.0f}\n"
                context += "\n"

        return context

    def ask_groq(self, question):
        """Main query function using Groq API"""
        try:
            # Build context with general knowledge
            context = self.build_context(question)

            # Create prompt
            full_prompt = f"""{context}

USER QUESTION: {question}

INSTRUCTIONS:
- Use the GENERAL KNOWLEDGE BASE to explain price movements even if specific date data is unavailable
- Provide educated analysis based on typical price dynamics and seasonal patterns
- If specific data is available, cite exact numbers and dates
- If specific data is NOT available, explain likely causes based on general agricultural market knowledge
- Mention research findings (weather impacts, fuel correlations, supply patterns) when relevant
- Be specific and informative, combining data-driven insights with domain knowledge
- Structure answers clearly with bullet points when helpful

ANSWER:"""

            # Call Groq API
            response = self.groq.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {
                        "role": "system",
                        "content": "You are an expert agricultural economist and data scientist specializing in carrot price forecasting in Sri Lanka. Provide insightful, data-driven answers combining specific data with general market knowledge."
                    },
                    {
                        "role": "user",
                        "content": full_prompt
                    }
                ],
                max_tokens=1500,
                temperature=0.7,
                top_p=0.9
            )

            # Extract answer
            answer = response.choices[0].message.content

            # Add footer
            tokens_used = response.usage.total_tokens
            answer += f"\n\n---\n*Powered by Llama 3.3 70B | {len(self.predictions)} days predictions"
            if self.original_data is not None:
                answer += f" | {len(self.original_data)} days full dataset"
            answer += f" | {tokens_used} tokens*"

            return answer

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}\n\n"

            if "rate_limit" in str(e).lower():
                error_msg += "⏱️ Rate limit reached. Please wait a moment and try again."
            elif "invalid" in str(e).lower() and "key" in str(e).lower():
                error_msg += "🔑 API key issue. Please check your Groq API key."
            else:
                error_msg += "Please check your internet connection and try again."

            return error_msg

# Initialize the agent with original dataset
agent = CarrotPriceAgent(groq_client, predictions_df, original_df)

print("="*60)
print("✅ AGENT INITIALIZED WITH GENERAL KNOWLEDGE!")
print("="*60)
print(f"Predictions loaded: {len(predictions_df)} days")
print(f"Models available: {len(agent.model_results)}")
if original_df is not None:
    print(f"Original dataset: {len(original_df)} records with {len(original_df.columns)} features")
print("General knowledge base: ✅ Loaded (agricultural economics, market dynamics)")
print("Agent ready to answer questions with domain expertise!")
print("="*60)

✅ AGENT INITIALIZED AND READY!
Predictions loaded: 180 days
Models available: 4
Agent ready to answer questions!


📋 Cell 5- test API connection


In [44]:
print("="*60)
print("🔍 TESTING GROQ API CONNECTION")
print("="*60)

try:
    # Simple test
    test_response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # NEW - Better & Faster!
        messages=[{"role": "user", "content": "Say 'Hello! API is working!'"}],
        max_tokens=50
    )

    print("✅ API Connection Successful!")
    print(f"Response: {test_response.choices[0].message.content}")
    print(f"Model: {test_response.model}")
    print(f"Tokens used: {test_response.usage.total_tokens}")
    print("\n🎉 Ready to create Gradio interface!")

except Exception as e:
    print(f"❌ API Test Failed: {e}")
    print("\nPlease check:")
    print("1. API key is correct")
    print("2. Internet connection is working")
    print("3. Get new key at: https://console.groq.com/keys")

🔍 TESTING GROQ API CONNECTION
✅ API Connection Successful!
Response: Hello! API is working!
Model: llama-3.3-70b-versatile
Tokens used: 50

🎉 Ready to create Gradio interface!


Cell 6 - gradio interface

In [ ]:
def chat_function(message, history):
    """Process user message"""
    try:
        response = agent.ask_groq(message)
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}\n\nPlease try rephrasing your question."

# Create Gradio Chat Interface
interface = gr.ChatInterface(
    fn=chat_function,
    title="🥕 Carrot Price Prediction AI Agent (Enhanced with Domain Knowledge)",
    description="""
    **Powered by Llama 3.3 70B with Agricultural Economics Knowledge Base**

    **Now answers with general agricultural market knowledge even without specific date data!**
    
    **Ask me about:**
    - 📅 **Specific prices:** *"What was the price on April 15, 2024?"*
    - 📈 **Price movements:** *"Why did prices spike between April 2-8?"*
    - 🌧️ **Weather impacts:** *"How does rainfall affect carrot prices?"*
    - ⛽ **Fuel effects:** *"Explain the relationship between diesel prices and carrot prices"*
    - 🏆 **Model performance:** *"Which model achieved the best MAPE?"*
    - 📊 **Feature importance:** *"What are the most important price predictors?"*
    - 📚 **Methodology:** *"How did you collect and engineer features?"*
    - 🔮 **General trends:** *"What causes price volatility in vegetable markets?"*
    """,
    examples=[
        "Why did prices increase between April 2-8, 2024?",
        "What was the carrot price on June 15, 2024?",
        "How does heavy rainfall in Nuwara Eliya affect carrot prices?",
        "Explain the fuel price impact on transportation costs",
        "Which model has the best MAPE score and why?",
        "What are the main factors causing price spikes?",
        "Compare Simple LSTM vs Bidirectional LSTM performance",
        "What is the typical price range for carrots in Dambulla market?",
        "How long does it take for weather to impact market prices?",
        "What happens to prices during festival seasons?",
        "Explain the research methodology and data sources"
    ],
    theme=gr.themes.Soft(),
    cache_examples=False,
    chatbot=gr.Chatbot(height=500)
)

print("="*60)
print("🚀 LAUNCHING ENHANCED GRADIO INTERFACE")
print("="*60)

# Launch with public shareable link
interface.launch(
    share=True,  # Creates public link
    debug=True,
    show_error=True
)

print("\n✅ Interface launched with general knowledge capabilities!")
print("📱 Use the public link above to share with others")
print("⏱️ Link expires in 72 hours")
print("\n🎯 Agent can now explain price movements using:")
print("   - Specific historical data (when available)")
print("   - General agricultural economics knowledge")
print("   - Weather-price relationships from research")
print("   - Fuel price impacts and transportation costs")
print("   - Seasonal patterns and market dynamics")

/tmp/ipython-input-4261779394.py:34: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500)
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


🚀 LAUNCHING GRADIO INTERFACE
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0daf223c2f39a45082.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
